In [ ]:
import os

from dotenv import load_dotenv

load_dotenv(os.path.join("..", ".env"), override=True)

%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="LangSmith now uses UUID v7", 
    category=UserWarning,
)

# 5. Enhanced Deep Agent — Full Architecture

## Overview

This notebook documents **every component** of our Deep Agent architecture and adds enhancements inspired by the official `deepagents` library, adapted for **Amazon Nova (Bedrock)**.

### Why from-scratch instead of the library?

The `deepagents` library is designed for **Anthropic/Claude**:
- `AnthropicPromptCachingMiddleware` — incompatible with Nova
- Default model hardcoded to `ChatAnthropic(claude-sonnet-4-5)`
- No built-in web search tools (our `tavily_search` is custom)

By building from-scratch, we get:
- ✅ Full Nova/Bedrock compatibility
- ✅ Custom web search with `tavily_search`
- ✅ Same architectural patterns (todos, files, subagents)
- ✅ Skills system implementation
- ✅ Complete control and understanding

---

## Architecture Map

### 🏗️ Agents

| Agent | Role | Model | Tools |
|-------|------|-------|-------|
| **Orchestrator (Main)** | Coordinates all work, talks to user | Nova Lite/Pro | `write_todos`, `read_todos`, `ls`, `read_file`, `write_file`, `edit_file`, `glob_files`, `grep_files`, `load_skill`, `task` |
| **Research Sub-agent** | Web research with context isolation | Nova Lite | `tavily_search`, `think_tool` |
| **General-purpose Sub-agent** | Any delegated task | Nova Lite | All tools from orchestrator |

### 🔧 Tools by Category

| Category | Tool | Description |
|----------|------|-------------|
| **Planning** | `write_todos` | Create/update TODO list with status tracking |
| **Planning** | `read_todos` | Read current TODO list to stay on track |
| **Files (Basic)** | `ls` | List all files in virtual filesystem |
| **Files (Basic)** | `read_file` | Read file with pagination (offset/limit) |
| **Files (Basic)** | `write_file` | Create or overwrite a file |
| **Files (Enhanced)** | `edit_file` | Find-and-replace in existing files |
| **Files (Enhanced)** | `glob_files` | Find files by pattern (`*.md`, `findings_*`) |
| **Files (Enhanced)** | `grep_files` | Search text across all files |
| **Skills** | `load_skill` | Load a SKILL.md with detailed instructions |
| **Research** | `tavily_search` | Web search + save results to files |
| **Research** | `think_tool` | Strategic reflection and planning |
| **Delegation** | `task` | Spawn isolated sub-agent for complex tasks |

### 📁 State Schema

```python
class DeepAgentState(AgentState):
    todos: list[Todo]        # Task planning and tracking
    files: dict[str, str]    # Virtual filesystem (key=path, value=content)
```

### 📋 TODO Item

```python
class Todo(TypedDict):
    content: str                                    # "Research MCP protocol"
    status: Literal["pending", "in_progress", "completed"]  # Current state
```

---

## What are Skills? (SKILL.md)

**Skills** are a concept from the Deep Agents framework for extending agent capabilities via **filesystem-based instruction files**.

### How they work:

1. A skill is a **directory** containing a `SKILL.md` file
2. `SKILL.md` has **YAML frontmatter** (name + description) and **markdown instructions**
3. The agent sees only the **name + description** initially (progressive disclosure)
4. When the agent decides a skill is relevant, it calls `load_skill(name)` to read the full content

### Example SKILL.md:

```yaml
---
name: web-research
description: Use this skill for research tasks requiring web searches.
---

# web-research

## Instructions
1. Plan your research queries
2. Execute searches with tavily_search
3. Reflect after each search using think_tool
4. Synthesize and deliver findings
```

### Why skills matter:
- Extend capabilities **without adding more tools**
- Save **tokens** through progressive disclosure
- Provide **domain-specific guidance** (research methodology, code review checklists, etc.)
- Easy to create — just write a markdown file!

> **Note:** The installed `deepagents` library does NOT include the `skills` parameter.
> It's a newer feature. Here we implement it **from-scratch**.

---

## Part 1: Setup & Model

In [ ]:
from datetime import datetime

from IPython.display import Image, display
from langchain_aws import ChatBedrockConverse

from utils import format_messages, show_prompt, stream_agent

# --- Model Setup ---
llm_nova_lite = ChatBedrockConverse(
    model="us.amazon.nova-2-lite-v1:0",
    region_name="us-east-1",
    temperature=0.0,
)

llm_nova_pro = ChatBedrockConverse(
    model="us.amazon.nova-2-pro-v1:0",
    region_name="us-east-1",
    temperature=0.0,
)

# Choose your model
model = llm_nova_lite
print(f"✅ Model loaded: {model.model}")

---

## Part 2: Import All Tools & Components

In [ ]:
# --- State ---
from deep_agents_from_scratch.state import DeepAgentState, Todo

# --- TODO Tools ---
from deep_agents_from_scratch.todo_tools import write_todos, read_todos

# --- Basic File Tools (from notebook 2) ---
from deep_agents_from_scratch.file_tools import ls, read_file, write_file

# --- Enhanced File Tools (NEW) ---
from deep_agents_from_scratch.enhanced_file_tools import edit_file, glob_files, grep_files

# --- Research Tools ---
from deep_agents_from_scratch.research_tools import tavily_search, think_tool

# --- Skills ---
from deep_agents_from_scratch.skills import (
    load_skill,
    discover_skills,
    get_skills_system_prompt,
    RESEARCH_SKILL_MD,
    CODE_REVIEW_SKILL_MD,
)

# --- Task Tool ---
from deep_agents_from_scratch.task_tool import _create_task_tool, SubAgent

# --- Prompts ---
from deep_agents_from_scratch.prompts import (
    RESEARCHER_INSTRUCTIONS,
    TODO_USAGE_INSTRUCTIONS,
    FILE_USAGE_INSTRUCTIONS,
    SUBAGENT_USAGE_INSTRUCTIONS,
)

print("✅ All components imported successfully")

---

## Part 3: Review the Enhanced Tools

### Enhanced File Tools (NEW)

These 3 tools are inspired by the `deepagents` library's `FilesystemMiddleware`:

In [ ]:
print("=" * 60)
print("ENHANCED FILE TOOLS")
print("=" * 60)

for tool_fn in [edit_file, glob_files, grep_files]:
    print(f"\n🔧 {tool_fn.name}")
    print("-" * 40)
    print(tool_fn.description[:200] + "...")
    print()

### Skills Tool (NEW)

In [ ]:
print("=" * 60)
print("SKILLS TOOL")
print("=" * 60)
print(f"\n🔧 {load_skill.name}")
print("-" * 40)
print(load_skill.description)

### Preview: Example Skills

In [ ]:
# Show what a SKILL.md looks like
print("📋 web-research SKILL.md:")
print("=" * 60)
print(RESEARCH_SKILL_MD[:500])
print("...")
print()

# Demonstrate progressive disclosure
from deep_agents_from_scratch.skills import parse_skill_md

parsed = parse_skill_md(RESEARCH_SKILL_MD)
print("🔍 Progressive Disclosure (what agent sees first):")
print(f"  Name: {parsed['name']}")
print(f"  Description: {parsed['description']}")
print(f"  Instructions length: {len(parsed['instructions'])} chars (loaded on demand)")

---

## Part 4: Build the Enhanced Agent

In [ ]:
from langchain.agents import create_agent

# --- All tools for the orchestrator ---
orchestrator_tools = [
    # Planning
    write_todos,
    read_todos,
    # Basic Files
    ls,
    read_file,
    write_file,
    # Enhanced Files (NEW)
    edit_file,
    glob_files,
    grep_files,
    # Skills (NEW)
    load_skill,
    # Research  
    think_tool,
]

# --- Sub-agents ---
subagents = [
    SubAgent(
        name="research-agent",
        description="Delegated research agent for complex web searches. Has tavily_search and think_tool.",
        prompt=RESEARCHER_INSTRUCTIONS.format(date=datetime.now().strftime("%a %b %-d, %Y")),
        tools=["tavily_search", "think_tool"],
    ),
]

# Add tavily_search to the tool list (for sub-agent assignment)
all_tools_for_registry = orchestrator_tools + [tavily_search]

# Create the task delegation tool
task_tool = _create_task_tool(
    tools=all_tools_for_registry,
    subagents=subagents,
    model=model,
    state_schema=DeepAgentState,
)

# Final tool list for orchestrator
final_tools = orchestrator_tools + [task_tool]

print(f"✅ Orchestrator tools: {len(final_tools)}")
for t in final_tools:
    print(f"   🔧 {t.name}")

In [ ]:
# --- Enhanced System Prompt ---
ENHANCED_SYSTEM_PROMPT = f"""You are a highly capable AI assistant with planning, research, file management, and skills capabilities.

{TODO_USAGE_INSTRUCTIONS}

{FILE_USAGE_INSTRUCTIONS}

{SUBAGENT_USAGE_INSTRUCTIONS.format(max_concurrent_research_units=3, max_researcher_iterations=3)}

## Skills System
You have access to a skills system. Skills are specialized instruction sets loaded from SKILL.md files.
- Use `ls()` to discover available skills in the filesystem
- Use `load_skill(name)` to read full instructions when a skill is relevant
- Skills provide step-by-step guidance for specific tasks (research, code review, etc.)
"""

print("System prompt length:", len(ENHANCED_SYSTEM_PROMPT), "chars")

In [ ]:
# --- Create the Enhanced Agent ---
enhanced_agent = create_agent(
    model,
    system_prompt=ENHANCED_SYSTEM_PROMPT,
    tools=final_tools,
    state_schema=DeepAgentState,
)

print("✅ Enhanced agent created!")
print(f"   Model: {model.model}")
print(f"   Tools: {len(final_tools)}")
print(f"   Sub-agents: {len(subagents)}")

---

## Part 5: Pre-load Skills into the Filesystem

We seed the virtual filesystem with our example SKILL.md files.
The agent will discover them when it calls `ls()`.

In [ ]:
# Pre-load skills into the virtual filesystem
initial_files = {
    "/skills/web-research/SKILL.md": RESEARCH_SKILL_MD,
    "/skills/code-review/SKILL.md": CODE_REVIEW_SKILL_MD,
}

# Verify skill discovery works
discovered = discover_skills(initial_files)
print("📋 Discovered Skills:")
for s in discovered:
    print(f"   • {s['name']}: {s['description'][:80]}...")

print()

# Show the skills system prompt that gets injected
skills_prompt = get_skills_system_prompt(initial_files)
print("📝 Skills System Prompt:")
print(skills_prompt)

---

## Part 6: Test Run — Research with Skills

Let's test the full enhanced agent. It should:
1. Call `ls()` to see the filesystem (including skills)
2. Create a TODO plan
3. Load relevant skills
4. Delegate research to the research sub-agent
5. Synthesize and respond

In [ ]:
# --- Test: Research Query ---
query = "Give me an overview of Model Context Protocol (MCP)."

initial_state = {
    "messages": [{"role": "user", "content": query}],
    "files": initial_files,  # Pre-load skills
}

# Run with streaming
result = await stream_agent(enhanced_agent, initial_state)

In [ ]:
# View final response
print("\n" + "=" * 60)
print("FINAL RESPONSE")
print("=" * 60)
print(result["messages"][-1].content)

In [ ]:
# View what files were created during the run
print("\n📁 Files in virtual filesystem:")
for path in result.get("files", {}).keys():
    print(f"   {path}")

# View TODOs
print("\n📋 Final TODOs:")
for todo in result.get("todos", []):
    status_emoji = {"pending": "⏳", "in_progress": "🔄", "completed": "✅"}
    emoji = status_emoji.get(todo["status"], "❓")
    print(f"   {emoji} {todo['content']} ({todo['status']})")

---

## Part 7: Test Enhanced File Tools

Let's verify the new `edit_file`, `glob_files`, and `grep_files` work.

In [ ]:
# Quick test of enhanced file tools (manual, outside agent)
from deep_agents_from_scratch.enhanced_file_tools import edit_file, glob_files, grep_files

test_files = {
    "notes.md": "# Notes\n\nHello World\nThis is a test.",
    "findings_mcp.md": "# MCP Findings\n\nMCP is a protocol for AI communication.",
    "findings_rag.md": "# RAG Findings\n\nRAG is retrieval augmented generation.",
    "readme.txt": "Just a readme file.",
}

# Test glob
print("🔍 glob_files('findings_*'):")
matches = glob_files.invoke({"pattern": "findings_*"}, config={"configurable": {"state": {"files": test_files, "messages": []}}})
print(f"   {matches}")

# Test grep
print("\n🔍 grep_files('protocol'):")
grep_result = grep_files.invoke(
    {"pattern": "protocol"}, 
    config={"configurable": {"state": {"files": test_files, "messages": []}}}
)
print(f"   {grep_result}")

---

## Summary: From-Scratch vs Library

### What we built (from-scratch, Nova compatible):

| Feature | From-scratch | Library (`deepagents`) |
|---------|-------------|------------------------|
| **TODO Tools** | `write_todos` + `read_todos` ✅ | `write_todos` only |
| **File Tools** | `ls`, `read_file`, `write_file`, `edit_file`, `glob_files`, `grep_files` ✅ | Same + `execute` |
| **Skills** | `load_skill` + SKILL.md parsing ✅ | Newer version only |
| **Research** | `tavily_search` + `think_tool` ✅ | ❌ Not included |
| **Sub-agents** | `task` tool ✅ | `task` tool (more complex middleware) |
| **Model** | Nova (Bedrock) ✅ | Claude (Anthropic) |
| **Context Management** | Virtual filesystem in state ✅ | Multiple backends (state/disk/store) |

### What the library has that we don't (yet):
- `execute` tool (sandbox command execution)
- `PatchToolCallsMiddleware` (fixing orphaned tool calls)
- `SummarizationMiddleware` (compressing long contexts)
- Multiple filesystem backends (disk, store, composite)